# ISA-API built-in semantic requirements for `protocol type` values

## Abstract:

In this notebook, we document the consequences of not using the right `protocol type` value in the context of specific ISA assays.

It is therefore important that developers and users of the ISA-API be aware of the defautl ISA configuration and the list of protocol types used in the assay workflow associated with a given ISA Assay type, which is defined by a combination of `Measurement Type` and `Technology Type`.

We have documented the full list of protocol types in the following document [link to document](https://www.todo.org)


## Let's get the tools

In [1]:
# If executing the notebooks on `Google Colab`,uncomment the following command 
# and run it to install the required python libraries. Also, make the test datasets available.

# !pip install -r requirements.txt

In [2]:
import os
import isatools
from isatools import isatab

from isatools.model import (
    Investigation,
    Study,
    OntologySource,
    OntologyAnnotation,
    Person,
    Publication,
    Source,
    Sample,
    Characteristic,
    Protocol,
    Process,
    Assay,
    Material,
    DataFile,
    batch_create_materials,
    plink
)
import json
from isatools.isajson import ISAJSONEncoder
from isatools.isatab import dump_tables_to_dataframes

## 1. Create an empty Investigation object and set some values to the instance variables.

In [3]:
investigation = Investigation()
# investigation.identifier = "i1"
# investigation.title = "My Simple ISA Investigation"
# investigation.description = "We could alternatively use the class constructor's parameters to set some default " \
#                             "values at the time of creation, however we want to demonstrate how to use the " \
#                             "object's instance variables to set values."
# investigation.submission_date = "2016-11-03"
# investigation.public_release_date = "201611-03"

# Create an empty Study object and set some values. The Study must have a filename, otherwise when we serialize it
# to ISA-Tab we would not know where to write it. We must also attach the study to the investigation by adding it
# to the 'investigation' object's list of studies.

study = Study(filename="s_study.txt")
study.identifier = "s1"
study.title = "My ISA Study"
study.description = "Like with the Investigation, we could use the class constructor to set some default values, " \
                    "but have chosen to demonstrate in this example the use of instance variables to set initial " \
                    "values."
study.submission_date = "2016-11-03"
study.public_release_date = "2016-11-03"
investigation.studies.append(study)

# Some instance variables are typed with different objects and lists of objects. For example, a Study can have a
# list of design descriptors. A design descriptor is an Ontology Annotation describing the kind of study at hand.
# Ontology Annotations should typically reference an Ontology Source. We demonstrate a mix of using the class
# constructors and setting values with instance variables. Note that the OntologyAnnotation object
# 'intervention_design' links its 'term_source' directly to the 'obi' object instance. To ensure the OntologySource
# is encapsulated in the descriptor, it is added to a list of 'ontology_source_references' in the Investigation
# object. The 'intervention_design' object is then added to the list of 'design_descriptors' held by the Study
# object.

obi = OntologySource(name='OBI', description="Ontology for Biomedical Investigations")
investigation.ontology_source_references.append(obi)
intervention_design = OntologyAnnotation(term_source=obi)
intervention_design.term = "intervention design"
intervention_design.term_accession = "http://purl.obolibrary.org/obo/OBI_0000115"
study.design_descriptors.append(intervention_design)

# Other instance variables common to both Investigation and Study objects include 'contacts' and 'publications',
# each with lists of corresponding Person and Publication objects.

contact = Person(first_name="Alice", last_name="Robertson", affiliation="University of Life", roles=[OntologyAnnotation(term='submitter')])
study.contacts.append(contact)
publication = Publication(title="Experiments with Elephants", author_list="A. Robertson, B. Robertson")
publication.pubmed_id = "12345678"
publication.doi = "10.1038/sdata.2016.18" #https://doi.org/10.144534/rmh0000008"
publication.status = OntologyAnnotation(term="published")
study.publications.append(publication)

## 2. Create the ISA Study graph

In [4]:
# To create the study graph that corresponds to the contents of the study table file (the s_*.txt file), we need
# to create a process sequence. To do this we use the Process class and attach it to the Study object's
# 'process_sequence' list instance variable. Each process must be linked with a Protocol object that is attached to
# a Study object's 'protocols' list instance variable. The sample collection Process object usually has as input
# a Source material and as output a Sample material.

# Here we create one Source material object and attach it to our study.

source = Source(name='source_material')
study.sources.append(source)

# Then we create three Sample objects, with organism as Homo Sapiens, and attach them to the study. We use the utility function
# batch_create_material() to clone a prototype material object. The function automatiaclly appends
# an index to the material name. In this case, three samples will be created, with the names
# 'sample_material-0', 'sample_material-1' and 'sample_material-2'.

prototype_sample = Sample(name='sample_material', derives_from=(source,))
ncbitaxon = OntologySource(name='NCBITaxon', description="NCBI Taxonomy")
characteristic_organism = Characteristic(category=OntologyAnnotation(term="Organism"),
                                 value=OntologyAnnotation(term="Homo Sapiens", term_source=ncbitaxon,
                                                          term_accession="http://purl.bioontology.org/ontology/NCBITAXON/9606"))
prototype_sample.characteristics.append(characteristic_organism)

study.samples = batch_create_materials(prototype_sample, n=3)  # creates a batch of 3 samples

Now we create a single Protocol object that represents our sample collection protocol, and attach it to the
study object. Protocols must be declared before we describe Processes, as a processing event of some sort
must execute some defined protocol. In the case of the class model, Protocols should therefore be declared
before Processes in order for the Process to be linked to one.

In [5]:
sample_collection_protocol = Protocol(name="sample collection",
                                      protocol_type=OntologyAnnotation(term="sample collection"))
study.protocols.append(sample_collection_protocol)
sample_collection_process = Process(executes_protocol=sample_collection_protocol)

Next, we link our materials to the Process. In this particular case, we are describing a sample collection
process that takes one source material, and produces three different samples.

(source_material)->(sample collection)->[(sample_material-0), (sample_material-1), (sample_material-2)]

In [6]:
for src in study.sources:
    sample_collection_process.inputs.append(src)
for sam in study.samples:
    sample_collection_process.outputs.append(sam)


Finally, attach the finished Process object to the study process_sequence. This can be done many times to describe multiple sample collection events.

In [7]:
study.process_sequence.append(sample_collection_process)

## 3. Building an Assay object and attach two protocols, extraction and sequencing.

In [8]:
# Next, we build n Assay object and attach two protocols, extraction and sequencing.

assay = Assay(filename="a_assay.txt")

extraction_protocol = Protocol(name='extraction', protocol_type=OntologyAnnotation(term="material extraction"))
study.protocols.append(extraction_protocol)

sequencing_protocol = Protocol(name='sequencing', protocol_type=OntologyAnnotation(term="material sequencing"))
study.protocols.append(sequencing_protocol)

# To build out assay graphs, we enumereate the samples from the study-level, and for each sample we create an
# extraction process and a sequencing process. The extraction process takes as input a sample material, and produces
# an extract material. The sequencing process takes the extract material and produces a data file. This will
# produce three graphs, from sample material through to data, as follows:
#
# (sample_material-0)->(extraction)->(extract-0)->(sequencing)->(sequenced-data-0)
# (sample_material-1)->(extraction)->(extract-1)->(sequencing)->(sequenced-data-1)
# (sample_material-2)->(extraction)->(extract-2)->(sequencing)->(sequenced-data-2)
#
# Note that the extraction processes and sequencing processes are distinctly separate instances, where the three
# graphs are NOT interconnected.

for i, sample in enumerate(study.samples):

    # create an extraction process that executes the extraction protocol

    extraction_process = Process(executes_protocol=extraction_protocol)

    # extraction process takes as input a sample, and produces an extract material as output

    extraction_process.inputs.append(sample)
    material = Material(name="extract-{}".format(i))
    material.type = "Extract Name"
    extraction_process.outputs.append(material)

    # create a sequencing process that executes the sequencing protocol

    sequencing_process = Process(executes_protocol=sequencing_protocol)
    sequencing_process.name = "assay-name-{}".format(i)
    sequencing_process.inputs.append(extraction_process.outputs[0])

    # Sequencing process usually has an output data file

    datafile = DataFile(filename="sequenced-data-{}".format(i), label="Raw Data File")
    sequencing_process.outputs.append(datafile)

    # Ensure Processes are linked forward and backward. plink(from_process, to_process) is a function to set
    # these links for you. It is found in the isatools.model package

    plink(extraction_process, sequencing_process)

    # make sure the extract, data file, and the processes are attached to the assay

    assay.data_files.append(datafile)
    assay.samples.append(sample)
    assay.other_material.append(material)
    assay.process_sequence.append(extraction_process)
    assay.process_sequence.append(sequencing_process)
    assay.measurement_type = OntologyAnnotation(term="genome sequencing")
    assay.technology_type = OntologyAnnotation(term="nucleotide sequencing")

We start by creation an ISA Assay Object and declaring the two protocols required by the workflow.

In [9]:
assay_1 = Assay(filename="a_assay_1.txt")

extraction_protocol = Protocol(name='extraction', protocol_type=OntologyAnnotation(term="material extraction"))
study.protocols.append(extraction_protocol)

sequencing_protocol_1 = Protocol(name='sequencing_1', protocol_type=OntologyAnnotation(term="nucleic acid sequencing"))
study.protocols.append(sequencing_protocol_1)

To build out assay graphs, we enumereate the samples from the study-level, and for each sample we create an
extraction process and a sequencing process. The extraction process takes as input a sample material, and produces
an extract material. The sequencing process takes the extract material and produces a data file. This will
produce three graphs, from sample material through to data, as follows:

`(sample_material-0)->(extraction)->(extract-0)->(sequencing)->(sequenced-data-0)`

`(sample_material-1)->(extraction)->(extract-1)->(sequencing)->(sequenced-data-1)`

`(sample_material-2)->(extraction)->(extract-2)->(sequencing)->(sequenced-data-2)`

:warning: 
Note that the extraction processes and sequencing processes are distinctly separate instances, where the three
graphs are NOT interconnected.

In [10]:
for i, sample in enumerate(study.samples):

    # create an extraction process that executes the extraction protocol

    extraction_process = Process(executes_protocol=extraction_protocol)

    # extraction process takes as input a sample, and produces an extract material as output

    extraction_process.inputs.append(sample)
    material = Material(name="extract-{}".format(i))
    material.type = "Extract Name"
    extraction_process.outputs.append(material)

    # create a sequencing process that executes the sequencing protocol

    sequencing_process = Process(executes_protocol=sequencing_protocol_1)
    sequencing_process.name = "assay-name-{}".format(i)
    sequencing_process.inputs.append(extraction_process.outputs[0])

    # Sequencing process usually has an output data file

    datafile = DataFile(filename="sequenced-data-{}".format(i), label="Raw Data File")
    sequencing_process.outputs.append(datafile)

    # Ensure Processes are linked forward and backward. plink(from_process, to_process) is a function to set
    # these links for you. It is found in the isatools.model package

    plink(extraction_process, sequencing_process)

    # make sure the extract, data file, and the processes are attached to the assay

    assay_1.data_files.append(datafile)
    assay_1.samples.append(sample)
    assay_1.other_material.append(material)
    assay_1.process_sequence.append(extraction_process)
    assay_1.process_sequence.append(sequencing_process)
    assay_1.measurement_type = OntologyAnnotation(term="genome sequencing")
    assay_1.technology_type = OntologyAnnotation(term="nucleotide sequencing")

In [11]:
# Next, we build n Assay objects and attach the protocols, extraction, sequencing and a data transformation step

assay_2 = Assay(filename="a_assay_2.txt")
extraction_protocol = Protocol(name='nucleic acid extraction', protocol_type=OntologyAnnotation(term="nucleic acid extraction"))
study.protocols.append(extraction_protocol)
sequencing_protocol = Protocol(name='sequencing_2', protocol_type=OntologyAnnotation(term="nucleic acid sequencing"))
study.protocols.append(sequencing_protocol)
dt_protocol = Protocol(name='analysis', protocol_type=OntologyAnnotation(term="sequence analysis data transformation"))
study.protocols.append(dt_protocol)

# To build out assay graphs, we enumereate the samples from the study-level, and for each sample we create an
# extraction process and a sequencing process. The extraction process takes as input a sample material, and produces
# an extract material. The sequencing process takes the extract material and produces a data file. This will
# produce three graphs, from sample material through to data, as follows:
#
# (sample_material-0)->(extraction)->(extract-0)->(sequencing)->(sequenced-data-0)
# (sample_material-1)->(extraction)->(extract-1)->(sequencing)->(sequenced-data-1)
# (sample_material-2)->(extraction)->(extract-2)->(sequencing)->(sequenced-data-2)
#
# Note that the extraction processes and sequencing processes are distinctly separate instances, where the three
# graphs are NOT interconnected.

for i, sample in enumerate(study.samples):

    # create an extraction process that executes the extraction protocol

    extraction_process = Process(executes_protocol=extraction_protocol)

    # extraction process takes as input a sample, and produces an extract material as output

    extraction_process.inputs.append(sample)
    material = Material(name="extract-{}".format(i))
    material.type = "Extract Name"
    extraction_process.outputs.append(material)

    # create a sequencing process that executes the sequencing protocol

    sequencing_process = Process(executes_protocol=sequencing_protocol)
    sequencing_process.name = "assay-name-{}".format(i)
    sequencing_process.inputs.append(extraction_process.outputs[0])

    # Sequencing process usually has an output data file

    datafile = DataFile(filename="sequenced-data-{}".format(i), label="Raw Data File")
    sequencing_process.outputs.append(datafile)
    
    # Data Analysis process
    dt_process = Process(executes_protocol=dt_protocol)
    dt_process.name = "dt-name-{}".format(i)
    dt_process.inputs.append(sequencing_process.outputs[0])
    
    derived_file = DataFile(filename="derived-data-{}".format(i), label="Derived Data File")
    dt_process.outputs.append(derived_file)
    
    # Ensure Processes are linked forward and backward. plink(from_process, to_process) is a function to set
    # these links for you. It is found in the isatools.model package

    plink(extraction_process, sequencing_process)
    plink(sequencing_process, dt_process)

    # make sure the extract, data file, and the processes are attached to the assay

    assay_2.data_files.append(datafile)
    assay_2.samples.append(sample)
    assay_2.other_material.append(material)
    assay_2.process_sequence.append(extraction_process)
    assay_2.process_sequence.append(sequencing_process)
    assay_2.process_sequence.append(dt_process)
    assay_2.measurement_type = OntologyAnnotation(term="transcription profiling")
    assay_2.technology_type = OntologyAnnotation(term="nucleotide sequencing")


print(assay_2)

Assay(
    measurement_type=transcription profiling
    technology_type=nucleotide sequencing
    technology_platform=
    filename=a_assay_2.txt
    data_files=3 DataFile objects
    samples=3 Sample objects
    process_sequence=9 Process objects
    other_material=3 Material objects
    characteristic_categories=0 OntologyAnnots
    comments=0 Comment objects
    units=0 Unit objects
)


In [12]:
# attach the assays to the study

study.assays.append(assay)
study.assays.append(assay_1)
study.assays.append(assay_2)

In [13]:
dataframes = dump_tables_to_dataframes(investigation)

2021-07-21 17:40:39,386 [INFO]: isatab.py(_all_end_to_end_paths:1131) >> [0]
2021-07-21 17:40:39,386 [WARNING]: isatab.py(write_study_table_files:1194) >> [5, 2, 3, 4, 0]
2021-07-21 17:40:39,387 [INFO]: isatab.py(_longest_path_and_attrs:1091) >> [[0, 5, 2], [0, 5, 3], [0, 5, 4]]
2021-07-21 17:40:39,426 [INFO]: isatab.py(_all_end_to_end_paths:1131) >> [2, 3, 4]
2021-07-21 17:40:39,426 [INFO]: isatab.py(_longest_path_and_attrs:1091) >> [[2, 6, 7, 8], [3, 10, 11, 12], [4, 14, 15, 16]]
2021-07-21 17:40:39,427 [INFO]: isatab.py(_longest_path_and_attrs:1091) >> [[2, 6, 7, 8], [3, 10, 11, 12], [4, 14, 15, 16]]
2021-07-21 17:40:39,435 [INFO]: isatab.py(_all_end_to_end_paths:1131) >> [2, 3, 4]
2021-07-21 17:40:39,436 [INFO]: isatab.py(_longest_path_and_attrs:1091) >> [[2, 18, 19, 20], [3, 22, 23, 24], [4, 26, 27, 28]]
2021-07-21 17:40:39,437 [INFO]: isatab.py(_longest_path_and_attrs:1091) >> [[2, 18, 19, 20], [3, 22, 23, 24], [4, 26, 27, 28]]
2021-07-21 17:40:39,446 [INFO]: isatab.py(_all_end_t

In [14]:
for key in dataframes.keys():
    display(key)

's_study.txt'

'a_assay_2.txt'

'a_assay.txt'

'a_assay_1.txt'

## Inspecting the different assay tables reveals the importance of using the right strings 

In [15]:
 dataframes['a_assay.txt']

,Sample Name,Protocol REF,Extract Name,Protocol REF.1,Raw Data File
0,sample_material-0,extraction,extract-0,sequencing,sequenced-data-0
1,sample_material-1,extraction,extract-1,sequencing,sequenced-data-1
2,sample_material-2,extraction,extract-2,sequencing,sequenced-data-2


In [16]:
 dataframes['a_assay_1.txt']

,Sample Name,Protocol REF,Extract Name,Protocol REF.1,Assay Name,Raw Data File
0,sample_material-0,extraction,extract-0,sequencing_1,assay-name-0,sequenced-data-0
1,sample_material-1,extraction,extract-1,sequencing_1,assay-name-1,sequenced-data-1
2,sample_material-2,extraction,extract-2,sequencing_1,assay-name-2,sequenced-data-2


In [17]:
 dataframes['a_assay_2.txt']

,Sample Name,Protocol REF,Extract Name,Protocol REF.1,Assay Name,Raw Data File,Protocol REF.2,Data Transformation Name,Derived Data File
0,sample_material-0,nucleic acid extraction,extract-0,sequencing_2,assay-name-0,sequenced-data-0,analysis,dt-name-0,derived-data-0
1,sample_material-1,nucleic acid extraction,extract-1,sequencing_2,assay-name-1,sequenced-data-1,analysis,dt-name-1,derived-data-1
2,sample_material-2,nucleic acid extraction,extract-2,sequencing_2,assay-name-2,sequenced-data-2,analysis,dt-name-2,derived-data-2


In [18]:
isatab.dump(investigation,'./notebook-output/isa-protocol-type-assay/') 

2021-07-21 17:40:39,603 [INFO]: isatab.py(_all_end_to_end_paths:1131) >> [0]
2021-07-21 17:40:39,604 [WARNING]: isatab.py(write_study_table_files:1194) >> [5, 2, 3, 4, 0]
2021-07-21 17:40:39,604 [INFO]: isatab.py(_longest_path_and_attrs:1091) >> [[0, 5, 2], [0, 5, 3], [0, 5, 4]]
2021-07-21 17:40:39,624 [INFO]: isatab.py(_all_end_to_end_paths:1131) >> [2, 3, 4]
2021-07-21 17:40:39,625 [INFO]: isatab.py(_longest_path_and_attrs:1091) >> [[2, 6, 7, 8], [3, 10, 11, 12], [4, 14, 15, 16]]
2021-07-21 17:40:39,625 [INFO]: isatab.py(_longest_path_and_attrs:1091) >> [[2, 6, 7, 8], [3, 10, 11, 12], [4, 14, 15, 16]]
2021-07-21 17:40:39,633 [INFO]: isatab.py(_all_end_to_end_paths:1131) >> [2, 3, 4]
2021-07-21 17:40:39,634 [INFO]: isatab.py(_longest_path_and_attrs:1091) >> [[2, 18, 19, 20], [3, 22, 23, 24], [4, 26, 27, 28]]
2021-07-21 17:40:39,634 [INFO]: isatab.py(_longest_path_and_attrs:1091) >> [[2, 18, 19, 20], [3, 22, 23, 24], [4, 26, 27, 28]]
2021-07-21 17:40:39,643 [INFO]: isatab.py(_all_end_t

isatools.model.Investigation(identifier='', filename='', title='', submission_date='', public_release_date='', ontology_source_references=[isatools.model.OntologySource(name='OBI', file='', version='', description='Ontology for Biomedical Investigations', comments=[])], publications=[], contacts=[], studies=[isatools.model.Study(filename='s_study.txt', identifier='s1', title='My ISA Study', description='Like with the Investigation, we could use the class constructor to set some default values, but have chosen to demonstrate in this example the use of instance variables to set initial values.', submission_date='2016-11-03', public_release_date='2016-11-03', contacts=[isatools.model.Person(last_name='Robertson', first_name='Alice', mid_initials='', email='', phone='', fax='', address='', affiliation='University of Life', roles=[isatools.model.OntologyAnnotation(term='submitter', term_source=None, term_accession='', comments=[])], comments=[])], design_descriptors=[isatools.model.Ontology

In [19]:
my_json_report = isatab.validate(open(os.path.join('./notebook-output/isa-protocol-type-assay/', 'i_investigation.txt')))

2021-07-21 17:40:39,664 [INFO]: isatab.py(validate:4206) >> Loading... ./notebook-output/isa-protocol-type-assay/i_investigation.txt
2021-07-21 17:40:39,778 [INFO]: isatab.py(validate:4208) >> Running prechecks...
2021-07-21 17:40:39,834 [WARNING]: isatab.py(check_doi:2176) >> (W) DOI 10.1038/sdata.2016.18 does not conform to DOI format
2021-07-21 17:40:39,835 [INFO]: isatab.py(validate:4229) >> Finished prechecks...
2021-07-21 17:40:39,836 [INFO]: isatab.py(validate:4230) >> Loading configurations found in /Users/philippe/.pyenv/versions/3.9.0/envs/isa-api-py39/lib/python3.9/site-packages/isatools/resources/config/xml
2021-07-21 17:40:39,857 [INFO]: isatab.py(validate:4235) >> Using configurations found in /Users/philippe/.pyenv/versions/3.9.0/envs/isa-api-py39/lib/python3.9/site-packages/isatools/resources/config/xml
2021-07-21 17:40:39,857 [INFO]: isatab.py(validate:4237) >> Checking investigation file against configuration...
2021-07-21 17:40:39,859 [WARNING]: isatab.py(check_secti

2021-07-21 17:40:40,142 [INFO]: utils.py(detect_isatab_process_pooling:85) >> Checking a_assay_1.txt
2021-07-21 17:40:40,143 [INFO]: utils.py(detect_isatab_process_pooling:85) >> Checking a_assay_2.txt
2021-07-21 17:40:40,143 [INFO]: utils.py(detect_graph_process_pooling:57) >> Possible process pooling detected on:  analysis
2021-07-21 17:40:40,144 [INFO]: utils.py(detect_graph_process_pooling:57) >> Possible process pooling detected on:  analysis
2021-07-21 17:40:40,145 [INFO]: utils.py(detect_graph_process_pooling:57) >> Possible process pooling detected on:  analysis
2021-07-21 17:40:40,145 [INFO]: isatab.py(validate:4444) >> Finished validation...


In [20]:
print(my_json_report)

{'errors': [], 'warnings': [{'message': 'DOI is not valid format', 'supplemental': 'Found 10.1038/sdata.2016.18 in DOI field', 'code': 3002}, {'message': 'A required property is missing', 'supplemental': 'A property value in Study Person Mid Initials of investigation file at column 1 is required', 'code': 4003}, {'message': 'A required column in assay table is not present', 'supplemental': "Required field 'Assay Name' not found in the file 'a_assay.txt'", 'code': 4010}, {'message': 'Missing Protocol declaration', 'supplemental': "Protocol(s) of type ['nucleic acid extraction'] defined in the ISA-configuration expected as a between 'Sample Name' and 'Extract Name' but has not been found, in the file 'a_assay.txt'", 'code': 1007}, {'message': 'Missing Protocol declaration', 'supplemental': "Protocol(s) of type ['library construction', 'nucleic acid sequencing'] defined in the ISA-configuration expected as a between 'Extract Name' and 'Raw Data File' but has not been found, in the file 'a

## About this notebook

- authors: philippe.rocca-serra@oerc.ox.ac.uk, massimiliano.izzo@oerc.ox.ac.uk
- license: CC-BY 4.0
- support: isatools@googlegroups.com
- issue tracker: https://github.com/ISA-tools/isa-api/issues